# Build a decision tree classifier (Information Gain-based) from scratch without using any machine learning libraries. Test your model on a simulated and real dataset.

BUILD A DECISION TREE CLASSIFIER AND TEST MODEL ON A REAL DATASET(HARE USE HEART DATASET)

In [1]:
import pandas as pd
import numpy as np
import math

In [3]:
# LOAD HEART DATASET

df = pd.read_csv(r'C:\Users\DEBABRATA\Desktop\Datasets\heart.csv')

In [4]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [5]:
df = df.sample(frac = 1)
header = list(df.columns)

In [6]:
# class_counts FUNCTION

def class_counts(rows,col):
    counts = {}
    for row in rows:
        label = row[col]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
# split FUNCTION

def split_d(D, attr,col):
    split = []
    for i in D:
        if i[col]==attr:
            split.append(i)
    return split

In [8]:
# info FUNCTION

def info(D):
    info_d=0
    unique_class = class_counts(D,-1)
    total_class = len(D)
    for i in unique_class:
        info_d+=(unique_class[i]/total_class)*abs(math.log((unique_class[i]/total_class),2))
    return info_d

In [9]:
# info_attr FUNCTION

def info_attr(D,attr):
    info_attr_d=0
    unique_lable = class_counts(traning_data,header.index(attr))
    total_lable = len(D)
    for i in unique_lable:
        info_attr_d+=(unique_lable[i]/total_lable)*info(split_d(D,i,header.index(attr)))
    return info_attr_d

In [10]:
# FINDING GAIN OF EACH ATTRIBUTE

def gain(D,arrt):
    return info(D) - info_attr(D,arrt)

In [11]:
# spliting_criteria FUNCTION

def spliting_criteria(rows):
    best_gain = 0
    best_attribute=None
    for i in header[:-1]:
        current_gain=gain(rows,i)
        if best_gain<=current_gain:
            best_gain=current_gain
            best_attribute=i
    return best_gain, best_attribute

In [12]:
#Leaf CLASS

class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows,-1)

In [13]:
# Decision_Node CLASS

class Decision_Node:
    def __init__(self,
                 attribute,
                 attribute_type,
                 true_branch,
                 false_branch):
        self.attribute = attribute
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.attribute_type = attribute_type

In [14]:
# partition FUNCTION

def partition(D,col):
    first_unique = list(class_counts(D,col))[0]
    true_rows = []
    false_rows=[]
    for i in D:
        if i[col]==first_unique:
            true_rows.append(i)
        else:
            false_rows.append(i)
    return true_rows,false_rows,first_unique

In [15]:
# build_tree FUNCTION

def build_tree(rows):
    gain, attr = spliting_criteria(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows,true_rows_value = partition(rows,header.index(attr))
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(attr,true_rows_value, true_branch, false_branch)

In [16]:
# print_tree FUNCTION

def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.attribute) + " == " + str(node.attribute_type))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [17]:
actual_data = df.to_dict('split')
actual_data= actual_data['data']
len80 = len(actual_data)*.80
traning_data = actual_data[:int(len(actual_data)*.80)]
test_data = actual_data[:int(len(actual_data)-len80)]

In [18]:
# classify FUNCTION

def classify(node,row):
    if isinstance(node, Leaf):
        return node.predictions
    if node.attribute_type == row[header.index(node.attribute)] :
        return classify( node.true_branch,row)
    else :
        return classify( node.false_branch,row)

In [19]:
# PRINT THE TREE

my_tree = build_tree(traning_data)
print_tree(my_tree)

chol == 254
--> True:
  thal == 3
  --> True:
    Predict {0: 4}
  --> False:
    Predict {1: 1}
--> False:
  chol == 409
  --> True:
    Predict {0: 1}
  --> False:
    chol == 275
    --> True:
      ca == 1
      --> True:
        Predict {0: 1}
      --> False:
        Predict {1: 1}
    --> False:
      chol == 203
      --> True:
        thal == 3
        --> True:
          Predict {0: 1}
        --> False:
          Predict {1: 1}
      --> False:
        chol == 394
        --> True:
          Predict {1: 1}
        --> False:
          chol == 141
          --> True:
            Predict {1: 1}
          --> False:
            chol == 325
            --> True:
              Predict {1: 2}
            --> False:
              chol == 193
              --> True:
                ca == 0
                --> True:
                  Predict {1: 1}
                --> False:
                  Predict {0: 1}
              --> False:
                chol == 309
                --> True

In [20]:
x_test = list(map(lambda x: x[:-1], test_data))
y_test = list(map(lambda x: x[-1], test_data))

In [21]:
test_x = test_data[1][:-1]

In [24]:
result = []
for i in x_test:
    result.append(list(classify(my_tree,i))[0])

In [25]:
# SHOW THE RESULT

result

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0]

BUILD A DECISION TREE CLASSIFIER AND TEST MODEL ON A SIMULATED DATASET(HARE A MADE A DATESET (i.e My_play dataset) AND APPLY DECISION TREE CLASSIFIER ON THIS DATASET)

In [26]:
# LOAD A SIMULATED DATASET(My_play.csv)

df = pd.read_csv(r'C:\Users\DEBABRATA\Desktop\Datasets\My_play.csv')
print(df.shape)

(14, 5)


In [27]:
df = df.sample(frac = 1)
header = list(df.columns)

In [28]:
# class_counts FUNCTION

def class_counts(rows,col):
    counts = {}
    for row in rows:
        label = row[col]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [29]:
#split FUNCTION

def split_d(D, attr,col):
    split = []
    for i in D:
        if i[col]==attr:
            split.append(i)
    return split

In [30]:
# info FUNCTION

def info(D):
    info_d=0
    unique_class = class_counts(D,-1)
    total_class = len(D)
    for i in unique_class:
        info_d+=(unique_class[i]/total_class)*abs(math.log((unique_class[i]/total_class),2))
    return info_d

In [31]:
# info_attr FUNCTION

def info_attr(D,attr):
    info_attr_d=0
    unique_lable = class_counts(traning_data,header.index(attr))
    total_lable = len(D)
    for i in unique_lable:
        info_attr_d+=(unique_lable[i]/total_lable)*info(split_d(D,i,header.index(attr)))
    return info_attr_d

In [32]:
# FINDING GAIN OF EACH ATTRIBUTE
def gain(D,arrt):
    return info(D) - info_attr(D,arrt)

In [33]:
# spliting_criteria FUNCTION

def spliting_criteria(rows):
    best_gain = 0
    best_attribute=None
    for i in header[:-1]:
        current_gain=gain(rows,i)
        if best_gain<=current_gain:
            best_gain=current_gain
            best_attribute=i
    return best_gain, best_attribute

In [34]:
# Leaf CLASS

class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows,-1)

In [35]:
# Decision_Node CLASS

class Decision_Node:
    def __init__(self,
                 attribute,
                 attribute_type,
                 true_branch,
                 false_branch):
        self.attribute = attribute
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.attribute_type = attribute_type

In [36]:
# partition FUNCTION

def partition(D,col):
    first_unique = list(class_counts(D,col))[0]
    true_rows = []
    false_rows=[]
    for i in D:
        if i[col]==first_unique:
            true_rows.append(i)
        else:
            false_rows.append(i)
    return true_rows,false_rows,first_unique

In [37]:
# build_tree FUNCTION

def build_tree(rows):
    gain, attr = spliting_criteria(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows,true_rows_value = partition(rows,header.index(attr))
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(attr,true_rows_value, true_branch, false_branch)

In [38]:
# print_tree FUNCTION

def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.attribute) + " == " + str(node.attribute_type))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [39]:
actual_data = df.to_dict('split')
actual_data= actual_data['data']
len80 = len(actual_data)*.80
traning_data = actual_data[:int(len(actual_data)*.80)]
test_data = actual_data[:int(len(actual_data)-len80)]

In [40]:
# classify function

def classify(node,row):
    if isinstance(node, Leaf):
        return node.predictions
    if node.attribute_type == row[header.index(node.attribute)] :
        return classify( node.true_branch,row)
    else :
        return classify( node.false_branch,row)

In [41]:
# PRINT THE TREE

my_tree = build_tree(traning_data)
print_tree(my_tree)

Outlook == Overcast
--> True:
  Predict {'Yes': 3}
--> False:
  Temperature == Mild
  --> True:
    Predict {'Yes': 3, 'No': 1}
  --> False:
    Temperature == Hot
    --> True:
      Predict {'No': 2}
    --> False:
      Windy == f
      --> True:
        Predict {'Yes': 1}
      --> False:
        Predict {'No': 1}


In [42]:
test_data

[['Overcast', 'Hot', 'High', 'f', 'Yes'],
 ['Rainy', 'Mild', 'Normal', 'f', 'Yes']]

In [43]:
x_test = list(map(lambda x: x[:-1], test_data))
y_test = list(map(lambda x: x[-1], test_data))

In [44]:
y_test

['Yes', 'Yes']

In [45]:
test_x = test_data[1][:-1]

In [46]:
result = []
for i in x_test:
    result.append(list(classify(my_tree,i))[0])

In [47]:
# SHOW THE RESULT

result

['Yes', 'Yes']